## NREL WIND Toolkit - HSDS Demo

This notebook demonstrates basic usage of the National Renewable Energy Laboratory (NREL) Wind Integration National Dataset (WIND) Toolkit data. The data is provided from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS).  
Please consult the README file for setup instructions prior to running this notebook.


In [ ]:
%matplotlib inline
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pyproj import Proj
import dateutil

## Basic Usage

In [ ]:
# Open the wind data "file"
# server endpoint, username, password is found via a config file
f = h5pyd.File("/nrel/wtk-us.h5")  

In [ ]:
list(f.attrs)  # list attributes belonging to the root group

In [5]:
f.attrs['history']   # attributes can be used to provide desriptions of the content

'Produced by 3TIER, Inc. under NREL subcontract AGV-2-22460-01'

## Datasets

In [ ]:
# list the datasets in the file
for k in f:
    dset = f[k]
    print(f"{dset.name} {dset.shape}")

In [ ]:
# get reference to windspeed_100m dataset
dset = f['windspeed_100m']
dset.shape

In [ ]:
dset.dtype

In [ ]:
dset.shape[0] * dset.shape[1] * dset.shape[2] * 4 * 10**-12  # aprox 1 TB per dataset!

In [ ]:
# the chunks property show how the dataset values are internally stored - in tiles of this shape
dset.chunks

In [ ]:
# approx 1.5 MB per chunk
dset.chunks[0] * dset.chunks[1] * dset.chunks[2] * 4 * 10**-6 

## Time-slicing

Get the datetime index from the server and convert to a pandas dataframe for convenience:

In [ ]:
dt = f["datetime"]
dt = pd.DataFrame({"datetime": dt[:]},index=range(0,dt.shape[0]))
dt['datetime'] = dt['datetime'].apply(dateutil.parser.parse)
dt.head()

Extract indexes for a particular span of time:

In [ ]:
twentyten = dt.loc[(dt.datetime >= '2010-01-01') & (dt.datetime < '2011-01-01')].index
twentyten

Or a particular date:

In [ ]:
timestep = dt.loc[dt.datetime == '2012-04-01 12:00:00'].index[0]
timestep

## Map Data

In [ ]:
%time data = dset[timestep,::8,::8]   # extract every 8th record at a particular time

In [ ]:
plt.title(str(dt.loc[timestep,"datetime"]))
plt.imshow(data, origin="lower")

In [ ]:
%time data = dset[timestep,1100:1400,2000:2400]  # full-resolution subset

In [ ]:
plt.imshow(data, origin="lower")

## Nearest Timeseries for given Lat/Lon

In [ ]:
# This function finds the nearest x/y indices for a given lat/lon.
# Rather than fetching the entire coordinates database, which is 500+ MB, this
# uses the Proj4 library to find a nearby point and then converts to x/y indices
def indicesForCoord(f, lat_index, lon_index):
    dset_coords = f['coordinates']
    projstring = """+proj=lcc +lat_1=30 +lat_2=60 
                    +lat_0=38.47240422490422 +lon_0=-96.0 
                    +x_0=0 +y_0=0 +ellps=sphere 
                    +units=m +no_defs """
    projectLcc = Proj(projstring)
    origin_ll = reversed(dset_coords[0][0])  # Grab origin directly from database
    origin = projectLcc(*origin_ll)
    
    coords = (lon_index,lat_index)
    coords = projectLcc(*coords)
    delta = np.subtract(coords, origin)
    ij = [int(round(x/2000)) for x in delta]
    return tuple(reversed(ij))

NewYorkCity = (40.7128, -74.0059)
NewYorkCity_idx = indicesForCoord( f, NewYorkCity[0], NewYorkCity[1] )

print("y,x indices for New York City: \t\t {}".format(NewYorkCity_idx))
print("Coordinates of New York City: \t {}".format(NewYorkCity))
print("Coordinates of nearest point: \t {}".format(f["coordinates"][NewYorkCity_idx[0]][NewYorkCity_idx[1]]))

In [ ]:
# Get the entire 2010 timeseries data for a point in NYC
%time tseries = dset[min(twentyten):max(twentyten)+1, NewYorkCity_idx[0], NewYorkCity_idx[1]]

In [ ]:
len(tseries)   # 1 years * 365 days * 24 hours 

In [ ]:
plt.plot(dt.iloc[twentyten,].datetime, tseries)
plt.ylabel("Windspeed at 100m (m/s)")
plt.title("NYC Windspeed in 2010")

In [ ]:
# Fetch full timeseries data for all seven years
LongsPeak_idx = indicesForCoord(f, 40.2549, -105.6160)
%time tseries = dset[:,LongsPeak_idx[0],LongsPeak_idx[1]]

In [ ]:
plt.plot(dt.datetime, tseries)
plt.ylabel("Windspeed at 100m (m/s)")
plt.title("Longs Peak Windspeed 2007-2013")

## Windspeed Statistics

In [ ]:
dt["windspeed"] = tseries
dt["year"] = dt["datetime"].apply(lambda x: x.year)
dt["month"] = dt["datetime"].apply(lambda x: x.month)
dt["day"] = dt["datetime"].apply(lambda x: x.day)
dt["hour"] = dt["datetime"].apply(lambda x: x.hour)

In [ ]:
agg = dt.groupby(["month","hour"]).mean()
agg = agg.reset_index().pivot(index="month",columns="hour",values="windspeed")
agg

In [ ]:
plt.imshow(agg)
plt.xlabel("Hour")
plt.ylabel("Month")
plt.title("12 x 24 Mean Windspeed (m/s)")
plt.colorbar()